In [ ]:
import sys
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import missingno as msno
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
dframe = pd.read_csv('../input/marvel-vs-dc-imdb-dataset/Marvel_DC_imdb.csv')

In [ ]:
dframe.head()

In [ ]:
dframe.isna().sum()

In [ ]:
dframe.shape

## Plot missing values 

In [ ]:
msno.matrix(dframe) #shows metascore and USA_Gross cloumns have almost all missing data 

In [ ]:
dframe = dframe.drop(columns=['USA_Gross', 'Metascore', 'Unnamed: 0'])

In [ ]:
dframe.head()

In [ ]:
dframe.info()

In [ ]:
dframe.describe()

In [ ]:
dframe = dframe.dropna(axis=0) #dropping all missing values from rows of dframe

In [ ]:
dframe.isna().sum()

In [ ]:
dframe.shape #we are left with 1247 rows, 443 rows has missing data 

In [ ]:
dframe.columns

In [ ]:
dframe.Rating.value_counts

In [ ]:
dframe.dtypes

## Word Cloud on Movies Column

In [ ]:
comment_words = ''
stopwords = set(STOPWORDS)
  
# iterate through the csv file
for val in dframe['Movie'].values:
      
    # typecaste each val to string
    val = str(val)
  
    #split the value
    tokens = val.split()
      
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
      
    comment_words += " ".join(tokens)+" "
    
    
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, 
                      background_color='black', colormap='Set2', 
                      collocations=False, stopwords = STOPWORDS).generate(comment_words)
  
#plot the WordCloud image                       
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad = 0)
  
plt.show()

In [ ]:
#dframe.plot(x = 'Year', y = 'IMDB_Score', ylim = ([0,10]), linewidth=2.5, figsize = (30,10))

# Pre-Processing

In [ ]:
def convert_runtime(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace('min','')
    return int(new_val)

In [ ]:
converted_RunTime = dframe['RunTime'].apply(convert_runtime)

In [ ]:
def convert_Year(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace('(','').replace(')', '').replace(' Video', '').replace(' TV Special','').replace('III ','').replace('–','').replace(' TV Movie','')
    return int(new_val)

In [ ]:
converted_Year = dframe['Year'].apply(convert_Year)

In [ ]:
converted_Year.value_counts

In [ ]:
dframe.columns

In [ ]:
def convert_score(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    
    return int(val)

In [ ]:
def convert_votes(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','')
    return int(new_val)

In [ ]:
converted_Votes = dframe['Votes'].apply(convert_votes)

In [ ]:
converted_IMDB_Score = dframe['IMDB_Score'].apply(convert_score)

In [ ]:
dframe.insert(8, "C_IMDB_Score", converted_IMDB_Score, True)
dframe.insert(1, "C_Year", converted_Year, True)
dframe.insert(3, "C_RunTime", converted_RunTime, True)
dframe.insert(9, "C_Votes", converted_Votes, True)

In [ ]:
dframe = dframe.drop(['IMDB_Score', 'Year', 'RunTime', 'Votes'], axis = 1)

In [ ]:
dframe.head()

In [ ]:
sns.catplot(x="Category", y="C_IMDB_Score", kind = 'bar', hue='Rating', palette="vlag", data=dframe)

In [ ]:
sns.set_style('ticks')
fig, ax = plt.subplots()
fig.set_size_inches(10, 7)
plt.ylim(0, 10)
plt.title("Distribution of Rating by IMDB Score, Separated by Category")
sns.boxenplot(x='Rating', y='C_IMDB_Score', data=dframe, palette='hls', hue='Category',linewidth=2.5)
plt.legend(loc='lower right', fancybox=True)
sns.despine()

In [ ]:
dframe.head()

In [ ]:
group_avg_rating = dframe['C_IMDB_Score'].groupby(dframe['Category'])

In [ ]:
group_avg_rating.mean() #average rating for DC is more than Marvel

In [ ]:
dframe.groupby(['Category']).size() #this is a bias dataset as there are only 257 Marvel movies in the cleaned dataset 

In [ ]:
group_avg_runtime = dframe['C_RunTime'].groupby(dframe['Category'])

In [ ]:
group_avg_runtime.mean()

In [ ]:
group_avg_Votes = dframe['C_Votes'].groupby(dframe['Category'])

In [ ]:
group_avg_Votes.mean()

In [ ]:
dframe_rating = dframe.groupby('C_IMDB_Score')

In [ ]:
dframe_rating.describe()

In [ ]:
def ranker(df):
    df['movie_imdb_score_rank'] = np.arange(len(df)) + 1
    return df

In [ ]:
dframe.sort_values('C_IMDB_Score', ascending=False, inplace=True)

In [ ]:
dframe = dframe.groupby('Category').apply(ranker)

In [ ]:
dframe.head()

In [ ]:
num_score = dframe['C_IMDB_Score'].value_counts()

In [ ]:
dframe[dframe.C_IMDB_Score ==9].head(len(num_score))

In [ ]:
dframe.info()

## Applying weighted Average 

In [ ]:
v = dframe['C_IMDB_Score']        # Number of votes for the movie
R = dframe['C_Votes']             # Vote average of a movie which is a number from 0 to 10
C = dframe['C_Votes'].mean()      # The mean vote across the whole report 
m = dframe['C_IMDB_Score'].quantile(0.7)

In [ ]:
dframe['Weighted_average'] = ((R*v)+(C*m))/(v+m)

In [ ]:
dframe.head()

In [ ]:
movie_rankings = dframe.sort_values('Weighted_average',ascending=False)
movie_rankings[['Movie','C_Votes','C_IMDB_Score', 'movie_imdb_score_rank','Weighted_average']]

In [ ]:
#Compare the top 6 movies 

weighted_average = dframe.sort_values('Weighted_average',ascending=False)
plt.figure(figsize=(7,5))
axis1 = sns.barplot(x = dframe['Weighted_average'].head(10),y = dframe['Movie'].head(10))

plt.title('Best Movies according to Average Vote given by Users',weight='bold')
plt.xlabel('Weighted Average Score',weight='bold')
plt.ylabel('Movie Title',weight='bold')